In [1]:
# Mean Reciprocal Rank

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

if "DATABRICKS_RUNTIME_VERSION" in os.environ and not 'installed_libs' in globals():
  #CUDA = 'cu121' 
  installed_libs = True
  
  
  !pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
  import torch
  #os.environ['TORCH'] = torch.__version__
  #print(torch.__version__)
  #torch_version = '2.0.0+cu118'
  
  #!pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
  !pip install torch_geometric
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
  #!pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  #!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
  !pip install sentence-transformers
  !pip install torcheval
  !pip install matplotlib
  !pip install pandas
  !pip install tensorboard
  
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
  ROOT_FOLDER = '/dbfs/FileStore/GraphNeuralNetworks/'
else:
  ROOT_FOLDER = ''

In [4]:
from learnings_sampler_v1 import get_datasets, uniform_hgt_sampler, get_minibatch_count, add_reverse_edge_original_attributes_and_label_inplace, get_hgt_linkloader, get_single_minibatch_count

train_data, val_data, test_data = get_datasets(get_edge_attr=False, filename=ROOT_FOLDER+'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt', filter_top_k=True, top_k=50)


/home/amos/mambaforge/envs/pyg_torch21/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


In [5]:
import torch
from models.TransE import TransE
from models.DistMult import DistMult
from models.HGT import HGT
import torch_geometric
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, gnn : torch.nn.Module, head :  torch.nn.Module, node_types, edge_types, ggn_output_dim, pnorm):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn
        
        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)
        
        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}
            
        # hidden channels should be the output dim of gnn
        if head=='TransE': 
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim, p_norm=pnorm)  # KGE head with loss function
        elif head=='DistMult':
            self.head = DistMult(len(node_types), len(edge_types) , ggn_output_dim, p_norm=pnorm)  # KGE head with loss function
        else:
            raise NotImplementedError
        
        self.gnn = gnn
        
    

    def forward(self, hetero_data1, target_edge_type, edge_label_index, edge_label, hetero_data2=None, get_head_fn='loss'):
        
        if hetero_data2 is not None:
            assert target_edge_type[0] != target_edge_type[2], 'when passing two data objects, the edge type has to contain two different node types'
            head_embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = self.gnn(hetero_data2.x_dict, hetero_data2.edge_index_dict)[target_edge_type[2]][edge_label_index[1,:]]
        else:
            assert target_edge_type[0] == target_edge_type[2], 'when passing one data object, the edge type has to contain the same node types'


            embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)
            head_embeddings = embeddings[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = embeddings[target_edge_type[2]][edge_label_index[1,:]]

        
        edgeindex = self.edgeindex_lookup[target_edge_type]
        if get_head_fn=='loss':
            loss = self.head.loss(head_embeddings, edgeindex.to(device), tail_embeddings, edge_label)
            return loss
        elif get_head_fn=='forward':
            return self.head.forward(head_embeddings, edgeindex.to(device), tail_embeddings)
    
    def get_embeddings(self, hetero_data, rel_type, node_type, first_n_nodes=None):
        # gives us the closeness of embeddings to the other side of the relationship, i.e. e + r or e - r
        if node_type == rel_type[0]:
            have_head_or_tail =='head'
        else:
            have_head_or_tail =='tail'
            
        mask = torch.zeros(hetero_data.x_dict[rel_type[0]].shape[0])
        mask[:first_n_nodes] = 1
        embeddings = self.gnn(hetero_data.x_dict, hetero_data.edge_index_dict)[node_type][mask]
        return self.head.get_embeddings(embeddings, rel_type, have_head_or_tail)
        
        

    
# out_channels = 256
# hidden_channels = 256
# num_heads = 8
# num_layers = 2
# head = 'TransE'



In [6]:
def get_model(model_folder):
    metadata = train_data.metadata()
    # add selfloops
    for node_type in train_data.node_types:
        metadata[1].append((node_type, 'self_loop', node_type))    
    
    # infer model parameters, dimensions batchsize etc from filename
    out_channels = int(model_folder.split('outchannels_')[1].split('_')[0])
    hidden_channels = int(model_folder.split('hiddenchannels_')[1].split('_')[0])
    num_heads = int(model_folder.split('numheads_')[1].split('_')[0])
    num_layers = int(model_folder.split('numlayers_')[1].split('/')[0])
    head = model_folder.split('head_')[1].split('_')[0]
    learning_rate = float(model_folder.split('lr')[1].split('_')[0])
    batch_size = int(model_folder.split('bs')[1].split('_')[0])
    num_neighbors = [int(x) for x in model_folder.split('neighbors_')[1].split('head')[0].strip('_').split('_')]
    if 'pnrom' in model_folder:
        pnorm = int(model_folder.split('pnrom')[1].split('_')[0])
    elif 'pnorm' in model_folder:
        pnorm = int(model_folder.split('pnorm')[1].split('_')[0])
    else:
        pnorm = 1
        
    gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=train_data.node_types, data_metadata=metadata)

    model = Model(gnn, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels, pnorm=pnorm)
    #torch_geometric.compile(model, dynamic=True)
    #model.load_state_dict(torch.load(model_folder)['model_state_dict'])
    return model

In [7]:
model_foldere5 = ROOT_FOLDER+'models/learningpeople_hgt_20231102_004035_lr2e-06_bs32_neighbors_133_1333_head_TransE_hiddenchannels_256_outchannels_256_numheads_8_numlayers_2/Ep0_model_samplesseen154400.pt'
modele5 = get_model(model_foldere5)

model_foldere4 = ROOT_FOLDER+'models/learningpeople_hgt_20231102_114410_lr0.0002_bs32_neighbors_133_1333_head_TransE_hiddenchannels_256_outchannels_256_numheads_8_numlayers_2/Ep1_model_samplesseen154400.pt'
modele4 = get_model(model_foldere4)

model_foldere4euclidean = ROOT_FOLDER+'models/learningpeople_hgt_20231102_170612_pnrom2_llr0.0002_bs32_neighbors_133_1333_head_TransE_hiddenchannels_256_outchannels_256_numheads_8_numlayers_2/Ep1_model_samplesseen154400.pt'
modele4euclidean = get_model(model_foldere4euclidean)

In [8]:
def init_model(model, model_folder):
    model.to(device)
    num_neighbors = [int(x) for x in model_folder.split('neighbors_')[1].split('head')[0].strip('_').split('_')]
    
    # init dimensions of model by training it
    from tqdm.auto import tqdm
    from datetime import datetime
    input_edgetype = ('people', 'rev_course_and_programs_student', 'courses_and_programs')
    add_reverse_edge_original_attributes_and_label_inplace(train_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=train_data[input_edgetype] )
    add_reverse_edge_original_attributes_and_label_inplace(val_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=val_data[input_edgetype] )

    val_sampler = get_hgt_linkloader(val_data, input_edgetype, 10, False, 'triplet', 1, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)

    learning_rate = 2e-4
    # torch get optimizer by string name
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #2e-15

    # train model

    model.eval()
    minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id, src_nodes, dst_nodes = next(iter(val_sampler))
    #optimizer.zero_grad()
    loss = model(minibatchpart1.to(device), input_edgetype, edge_label_index.to(device), edge_label.to(device), minibatchpart2.to(device))
    model.load_state_dict(torch.load(model_folder)['model_state_dict'])
    model.to('cpu')
    

In [9]:
init_model(modele5, model_foldere5)
init_model(modele4, model_foldere4)
init_model(modele4euclidean, model_foldere4euclidean)

In [10]:
# evaluate model on test set
# init dimensions of model by training it
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np 

def evaluate(model, n_negatives, model_folder, on='test'):
    num_neighbors = [int(x) for x in model_folder.split('neighbors_')[1].split('head')[0].strip('_').split('_')]
    
    model.to(device)
    mrrs = []
   
    
    #test_sampler = get_hgt_linkloader(test_data, input_edgetype, 1, False, 'triplet', n_negatives, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)
    



    # test data
    train_data_text, val_data_text, test_data_text = get_datasets(get_edge_attr=False, filename=ROOT_FOLDER+'HeteroData_Learnings_normalized_triangles_withadditionaldata_v1.pt', filter_top_k=True, top_k=50, remove_text_attr=False)
    input_edgetype = ('people', 'rev_course_and_programs_student', 'courses_and_programs')
    if on=='test':
        add_reverse_edge_original_attributes_and_label_inplace(test_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=test_data[input_edgetype] )
        test_sampler = get_hgt_linkloader(test_data, input_edgetype, 1, False, 'triplet', n_negatives, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)
    elif on =='train':
        add_reverse_edge_original_attributes_and_label_inplace(train_data['courses_and_programs', 'course_and_programs_student', 'people'], reverse_edge=train_data[input_edgetype] )
        test_sampler = get_hgt_linkloader(train_data, input_edgetype, 1, False, 'triplet', n_negatives, num_neighbors, num_workers=0, prefetch_factor=None, pin_memory=True)
    # test data
    best_mrr, best_differences, best_src_nodes, best_dst_nodes = 0, None, None, None
    for i, minibatch in tqdm(enumerate(test_sampler)):
        if i==100:
            break
        model.eval()
        minibatchpart1, minibatchpart2, edge_label_index, edge_label, input_edge_id, global_src_nodes, global_dst_nodes = minibatch
        #optimizer.zero_grad()
        differences = model.forward(minibatchpart1.to(device), input_edgetype, edge_label_index.to(device), edge_label.to(device), minibatchpart2.to(device), get_head_fn='forward')
        #loss.backward()

        # define mrr with differences and labels
        # get rank of positive edge from tensor, positive edge is first in batch

        differences = -1* differences.cpu().detach().numpy()
        edge_label = edge_label.cpu().detach().numpy()
        rank = (differences < differences[0]).sum()

        # reciprocal
        mrr = 1/(rank+1)
        if mrr > best_mrr:
            best_mrr = mrr
            best_differences = differences
            best_src_nodes = global_src_nodes
            best_dst_nodes = global_dst_nodes
            print('new best mrr', best_mrr)
            
    
        mrrs.append(mrr)

        
        if False:
            jobpeople = test_data_text['jobs', 'job_student', 'people']
            job_edge = jobpeople.edge_index[1,:] == global_src_nodes[0]
            
            
            if torch.sum(job_edge) != 0:
                job_idx = jobpeople.edge_index[0,:][job_edge]
                
            else:
                job_idx = torch.tensor([0])

            job_edge = jobpeople.edge_label_index[1,:] == global_src_nodes[0]
            if torch.sum(job_edge) != 0:
                job_idx2 = jobpeople.edge_label_index[0,:][job_edge]
                torch.cat((job_idx, job_idx2)).squeeze()
                
                
            
        
            print('==============')
            print('==============')
            print('==============')
            # (differences < differences[0])
            print('mrr',mrr)
            print('rank',rank)
            for idx in job_idx:
                jobtitle = test_data_text['jobs'].TITLE[idx.item()]
                print('job title', jobtitle)
                
            print('original', test_data_text['courses_and_programs'].TITLE[global_dst_nodes[0]])
            print('original', test_data_text['courses_and_programs'].DESCRIPTION[global_dst_nodes[0]])
            
            # first 10 higher ranked
            mask = torch.where(torch.tensor((differences < differences[0])))
            indices = torch.argsort(torch.tensor(differences[mask]))
            for i in range(10):
                index = indices[i]
                global_index = global_dst_nodes[mask][index]
                print('------')
                print(differences[mask][index])
                print(test_data_text['courses_and_programs'].TITLE[global_index])
                print(test_data_text['courses_and_programs'].DESCRIPTION[global_index])

    print('mean mrr',np.mean(mrrs))
    print('mean rank',1/np.mean(mrrs))
    # mean rank
    model.to('cpu')


In [11]:
evaluate(modele4euclidean, 30000, model_foldere4euclidean)

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


1it [00:09,  9.64s/it]

new best mrr 0.00038774718883288094


3it [00:18,  5.82s/it]

new best mrr 0.05


4it [00:25,  6.09s/it]

new best mrr 0.0625


7it [00:40,  5.09s/it]

new best mrr 1.0


78it [05:25,  3.95s/it]

In [ ]:
evaluate(modele5, 50000, model_foldere5)

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


1it [00:07,  7.80s/it]

new best mrr 8.224360555966774e-05


2it [00:12,  5.95s/it]

new best mrr 0.000363901018922853


...

new best mrr 0.07692307692307693


525it [51:24,  5.89s/it]

new best mrr 0.2


604it [59:49,  5.94s/it]


KeyboardInterrupt: 

In [ ]:
evaluate(modele4, 50000, model_foldere4)

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


1it [00:04,  4.58s/it]

new best mrr 0.0006830601092896175
0.0006830601092896175


2it [00:08,  4.33s/it]

0.00013099292638197538


...

0.005076142131979695


100it [06:18,  3.94s/it]

0.0007987220447284345


100it [06:20,  3.80s/it]

mean mrr 0.08684316804801107
mean rank 11.515010592970906


In [ ]:
evaluate(modele5, 50000, model_foldere5, on='train')

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


1it [00:03,  3.09s/it]

new best mrr 0.00011668611435239207
0.00011668611435239207


2it [00:05,  2.85s/it]

new best mrr 0.0023148148148148147
0.0023148148148148147


...

0.0005497526113249038


100it [05:17,  3.78s/it]

0.00031426775612822125


100it [05:19,  3.19s/it]

mean mrr 0.004016088209092868
mean rank 248.99851495688998


In [ ]:
evaluate(modele4, 50000, model_foldere4, on='train')

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


1it [00:04,  4.12s/it]

new best mrr 8.352125615969264e-05
8.352125615969264e-05


2it [00:07,  3.64s/it]

new best mrr 0.024390243902439025
0.024390243902439025


...

0.0625


100it [05:57,  3.22s/it]

0.00025614754098360657


100it [05:58,  3.58s/it]

mean mrr 0.11484138897390082
mean rank 8.70766201049051


In [ ]:


#rank + 1 = 1/mmr
1/best_mrr -1

97.99999999999999

In [ ]:
edge_label

size of dataset on disk:  2.279761238 gb
loading saved heterodata object
for skill job edges keep top k edges per job, k is  50
keep tensor(1208056) of total 16289586


{'edge_index': tensor([[237571, 232648, 223362,  ..., 144382, 146638, 167762],
        [ 33929,  15584,  28839,  ...,  27864,  27235,  50546]]), 'edge_attr': tensor([[0.8948, 0.8944, 0.0037],
        [0.9398, 0.9394, 0.0038],
        [0.6720, 0.6716, 0.0036],
        ...,
        [0.5501, 0.5497, 0.0036],
        [0.6741, 0.6737, 0.0036],
        [0.9708, 0.9704, 0.0037]], dtype=torch.float64), 'edge_label': tensor([1., 1., 1.,  ..., 0., 0., 0.]), 'edge_label_index': tensor([[229076, 231606, 159751,  ..., 145961,  84933,  77255],
        [ 29467,  47061,  25464,  ...,  15266,  48246,  13604]])}

In [ ]:
best_src_nodes

tensor([253238])

In [ ]:
torch.sum(job_title)

tensor(1)

In [ ]:
test_data_text['people']

{'IS_ACTIVE_INFERRED': array(['Y', 'Y', 'N', ..., 'N', 'Y', 'N'], dtype=object), 'x': tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00, 1.8083e-03,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00, 1.8083e-03,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.0000e+00, 1.8083e-03,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 9.0416e-04,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 9.0416e-04,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 9.0416e-04,
         0.0000e+00]])}

In [ ]:
# (differences < differences[0])
print('original', test_data_text['courses_and_programs'].TITLE[best_dst_nodes[0]])
print('original', test_data_text['courses_and_programs'].DESCRIPTION[best_dst_nodes[0]])

# first 10 higher ranked
mask = torch.where(torch.tensor((best_differences < best_differences[0])))
indices = torch.argsort(torch.tensor(best_differences[mask]))
for i in range(10):
    index = indices[i]
    global_index = best_dst_nodes[mask][index]
    print('------')
    print(best_differences[mask][index])
    print(test_data_text['courses_and_programs'].TITLE[global_index])
    print(test_data_text['courses_and_programs'].DESCRIPTION[global_index])

original 4. Pre-Fund Maintenance on TLS
original 
This course provides information to perform pre-fund maintenance functions on the Term Lending System (TLS), including screen index organization, key features of selected screens and interactive customer scenarios. Estimated completion time is 30 minutes.


In [ ]:
# first 10 higher ranked
mask = torch.where(torch.tensor((best_differences < best_differences[0])))
indices = torch.argsort(torch.tensor(best_differences[mask]))
for i in range(10):
    index = indices[i]
    global_index = best_dst_nodes[mask][index]
    print('------')
    print(best_differences[mask][index])
    print(test_data_text['courses_and_programs'].TITLE[global_index])
    print(test_data_text['courses_and_programs'].DESCRIPTION[global_index])

------
0.6546815
Check-Off Request: Small Business Administration for AMSB Trainee

The CareerStart certfications on My Learning Centre have been set up in a unique way to make it easier and less time consuming to manage training completions. Under most CareerStart certifications there are two modules, 'Certification Check-off' and 'Courses'. The Courses are optional and can be taken by the Trainee if it has been deemed necessary to close a learning gap. The Certification Check-off is required, and whether training is completed or not, once it has been determined that the relevant skilland knowledge has been mastered, the Certification Check-off is used to show completion on MLC.
------
0.6613003
U.S. GCM Education Series Prime Services
Please join the next edition of the U.S. GCM Education Series to help the development and education of Scotiabankers. The next session topic is about Prime Services
------
0.67238903
Behavior: Putting Your Best Foot Forward

------
0.673694
Report of As

In [ ]:
test_data['